## Required Installation

In [1]:
!pip install auto_gptq
!pip install optimum
!pip install -U accelerate bitsandbytes datasets peft transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 77.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 96.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 10.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 108.3 MB/s eta 0:00:0000:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successf

In [2]:
from transformers.utils import is_auto_gptq_available,   is_optimum_available
print(is_auto_gptq_available())
print(is_optimum_available())

True
True


In [3]:
!pip install datasets==2.15.0

!pip install trl

!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/

!pip install ipywidgets==8.1.5

!pip show optimum

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 12.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
 

# Load model and tokenizer from huggingface

In [ ]:
!huggingface-cli login --token --Your Hugging-Face Key

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `Shadman114` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Shadman114`


In [5]:
from peft import prepare_model_for_kbit_training

from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig

#model_id = "TheBloke/Llama-2-7b-Chat-GPTQ"
model_id = "marcsun13/gemma-2-9b-it-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# Load RAFT dataset

In [6]:
from datasets import Dataset

dataset_path = "/kaggle/input/raft-train-dataset/data-00000-of-00001.arrow"

ds = Dataset.from_file(dataset_path)

In [7]:
ds

Dataset({
    features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction'],
    num_rows: 240
})

In [8]:
import pandas as pd

df = pd.DataFrame(ds)

df.head()

,id,type,question,context,oracle_context,cot_answer,instruction
0,seed_task_0,general,What is a Megger?,{'sentences': [['The word “Megger” is a regist...,The word “Megger” is a registered trademark\nW...,"assistant: To answer the question ""What is a M...",<DOCUMENT>The word “Megger” is a registered tr...
1,seed_task_1,general,It seems there was no specific context provide...,"{'sentences': [['it includes inspection, test...",,assistant: Since there is no specific context ...,"<DOCUMENT>it includes \ninspection, testing, o..."
2,seed_task_2,general,What is the purpose of circuit breaker testing?,{'sentences': [['CirCuiT Breaker TesTing guide...,CirCuiT Breaker TesTing guide 3,assistant: To answer the question about the pu...,<DOCUMENT>CirCuiT Breaker TesTing guide 3</DO...
3,seed_task_3,general,What are the main types of circuit breakers me...,{'sentences': [['Of interest is the curve [4-...,Contents\nIntroduction ..........................,assistant: To answer the question about the ma...,<DOCUMENT>Of interest is the curve [4-5]. As ...
4,seed_task_4,general,What are some error sources in testing?,{'sentences': [['Good to know about error sour...,Good to know about error sources ............ ...,assistant: To answer the question about error ...,<DOCUMENT>Good to know about error sources ......


In [9]:
# Combine 'user' and 'assistant' columns into a new 'messages' column as list of dictionaries

df['messages'] = df.apply(lambda row: [{'content': row['instruction'], 'role': 'user'},

                                       {'content': row['cot_answer'], 'role': 'assistant'}], axis=1)


In [10]:
import datasets

ds = datasets.Dataset.from_pandas(df)

ds

Dataset({
    features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'messages'],
    num_rows: 240
})

# Chat Template for prompt

In [11]:
# Set chat template

DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"

tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

In [12]:
import re

import random

from multiprocessing import cpu_count



def apply_chat_template(example, tokenizer):

    messages = example["messages"]

    # We add an empty system message if there is none

    if messages[0]["role"] != "system":

        messages.insert(0, {"role": "system", "content": ""})

    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)



    return example



column_names = list(ds.features)

raw_datasets = ds.map(apply_chat_template,

                                num_proc=cpu_count(),

                                fn_kwargs={"tokenizer": tokenizer},

                                remove_columns=column_names,

                                desc="Applying chat template",)

Applying chat template (num_proc=4):   0%|          | 0/240 [00:00<?, ? examples/s]

In [13]:
raw_datasets

Dataset({
    features: ['text'],
    num_rows: 240
})

In [14]:
raw_datasets["text"][0]

'<|system|>\n<eos>\n<|user|>\n<DOCUMENT>The word “Megger” is a registered trademark\nWWW.MEGGER.COM\n▪▪ Circuit breaker function and test methods\n▪▪ Practical circuit breaker testing\n▪▪ Megger CB testing products overview\nCircuit Breaker \ntesting guide</DOCUMENT>\n<DOCUMENT>over time will fi ll-up with water or water vapor. Hence, water filled v oids in the \nextruded insulation are known as water trees because th ese voids when inspected \nunder a microscope resemble like a tree . Studies indicated that water treeing is the \ncrucial degradation form that may affl ict older XLPE and high -molecular weight PE -\nextruded cables. Water treeing is self-propagating dendritic pattern of electro -\noxidation, which decreases extruded insulation AC and impulse breakdown \nstrengths. It is the main degradation mechanism of extruded medium-voltage power \ncables. Even though carefully studied, the initiation and growth mechanisms of water \ntreeing are not yet clear. </DOCUMENT>\nWhat is a

## Create train/test split

In [15]:
raw_datasets = raw_datasets.train_test_split(test_size=0.1)

# create the splits

train_dataset = raw_datasets["train"]

eval_dataset = raw_datasets["test"]


# Peft config

In [16]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(

    r=8,

    lora_alpha=32,

    target_modules=["k_proj","o_proj","q_proj","v_proj"],

    lora_dropout=0.05,

    bias="none",

    task_type="CAUSAL_LM"

)

# Quantization Config

In [17]:
from transformers import GPTQConfig



quantization_config = GPTQConfig(bits=4, disable_exllama=True)



model_kwargs = dict(

    torch_dtype="auto",

    use_cache=False, # set to False as we're going to use gradient checkpointing

    device_map="auto",

    quantization_config=quantization_config,

)


Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


# SFT Training

In [19]:


from trl import SFTTrainer

from peft import LoraConfig

from transformers import TrainingArguments



# path where the Trainer will save its checkpoints and logs

output_dir = 'raft-sft-output'



# based on config

training_args = TrainingArguments(

    fp16=True, # specify bf16=True instead when training on GPUs that support bf16

    do_eval=True,

    evaluation_strategy="epoch",

    gradient_accumulation_steps=128,

    gradient_checkpointing=True,

    gradient_checkpointing_kwargs={"use_reentrant": False},

    learning_rate=2.0e-05,

    log_level="info",

    logging_steps=5,

    logging_strategy="steps",

    lr_scheduler_type="cosine",

    max_steps=-1,

    num_train_epochs=4,

    output_dir=output_dir,

    overwrite_output_dir=True,

    per_device_eval_batch_size=1, # originally set to 8

    per_device_train_batch_size=1, # originally set to 8

    save_strategy="no",

    save_total_limit=None,

    seed=42,

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
trainer = SFTTrainer(

        model=model_id,

        #model_init_kwargs=model_kwargs,

        args=training_args,

        train_dataset=train_dataset,

        eval_dataset=eval_dataset,

        #dataset_text_field="text",

        tokenizer=tokenizer,

        # packing=True,

        peft_config=peft_config,

        # max_seq_length=tokenizer.model_max_length,

        # max_seq_length=512,

        #max_seq_length=2048

    )

<ipython-input-20-9b20d650f9d0>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/134k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

Converting eval dataset to ChatML:   0%|          | 0/24 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/24 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/24 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/24 [00:00<?, ? examples/s]

You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
Using auto half precision backend
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Setting pad_token=eos_token

In [21]:
tokenizer.pad_token = tokenizer.eos_token

## Start Training

In [22]:
!pip install transformers accelerate peft datasets trl

import json
from transformers import TrainingArguments, TrainerCallback, TrainerState, TrainerControl
from transformers.integrations import TensorBoardCallback

class CustomTensorBoardCallback(TensorBoardCallback):
    """
    Custom TensorBoard callback that handles non-serializable objects in TrainingArguments.
    """
    def on_train_begin(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        """
        Event called at the beginning of training.

        Overridden to handle non-serializable objects in TrainingArguments.
        """
        if self.tb_writer is not None:
            # Convert args to a dictionary and remove non-serializable objects
            args_dict = args.to_dict()

            # Iterate through the dictionary and remove non-serializable objects
            for key, value in list(args_dict.items()):
                if not isinstance(value,(int,float,str,bool,list,tuple,dict,type(None))):
                    del args_dict[key]

            self.tb_writer.add_text("args", json.dumps(args_dict, indent=2))

            if "model" in kwargs:
                model = kwargs["model"]
                if hasattr(model, "config") and model.config is not None:
                    model_config_json = model.config.to_json_string()
                    self.tb_writer.add_text("model_config", model_config_json)


# Replace the default TensorBoardCallback with the custom one
trainer.remove_callback(TensorBoardCallback)  # Remove default if present
trainer.add_callback(CustomTensorBoardCallback)  # Add custom callback

# Wandb  168899a11d1a94cd8fd80d8f2d31623ab2a485ab

In [24]:
import wandb
import os
os.environ["WANDB_API_KEY"] = "168899a11d1a94cd8fd80d8f2d31623ab2a485ab"

In [26]:
import wandb
wandb.login(key="168899a11d1a94cd8fd80d8f2d31623ab2a485ab")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [27]:
train_result = trainer.train()

The following columns in the training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 216
  Num Epochs = 4
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 128
  Total optimization steps = 4
  Number of trainable parameters = 8,945,664
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


Epoch,Training Loss,Validation Loss
1,No log,2.451845
2,No log,2.437057


The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 24
  Batch size = 1
The following columns in the evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 24
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)




In [28]:
 metrics = train_result.metrics

In [29]:
print(metrics)

{'train_runtime': 3874.9089, 'train_samples_per_second': 0.223, 'train_steps_per_second': 0.001, 'total_flos': 17547070894080.0, 'train_loss': 2.418092727661133}


## Save the finetuned_raft_model

In [30]:
output_dir_new = "finetuned_raft_model"

trainer.save_model(output_dir_new)

Saving model checkpoint to finetuned_raft_model
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--marcsun13--gemma-2-9b-it-GPTQ/snapshots/9c51e21691674594afc671d8edaeb93aa0cc9f7a/config.json
Model config Gemma2Config {
  "_name_or_path": "google/gemma-2-9b-it-gptq",
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": 1,
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "gemma2",
  "num_attention_heads": 16,
  "num_hidden_layers": 42,
  "num_key_value_heads": 8,
  "pad_token_id": 0,
  "quantization_config": {
    "batch_size": 1,
    "bits": 4,
    "block_name_to_quantize": 

In [6]:
import shutil

# Define the directory to be zipped and the output zip file name
directory_to_zip = '/kaggle/working/'
output_zip_file = '/kaggle/working/working_directory.zip'

# Zip the directory
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', directory_to_zip)

'/kaggle/working/working_directory.zip'

# Inference

## Load model

In [31]:
output_dir_new = "finetuned_raft_model"

In [6]:
# import os

# # Function to calculate the size of a folder
# def get_folder_size(folder_path):
#     total_size = 0
#     for dirpath, dirnames, filenames in os.walk(folder_path):
#         for f in filenames:
#             fp = os.path.join(dirpath, f)
#             total_size += os.path.getsize(fp)
#     return total_size

# # Path to the working directory
# working_dir = '/kaggle/working/'

# # Iterate over each subfolder in the working directory
# for subfolder in os.listdir(working_dir):
#     subfolder_path = os.path.join(working_dir, subfolder)
#     if os.path.isdir(subfolder_path):  # Only process directories
#         size_in_bytes = get_folder_size(subfolder_path)
#         size_in_mb = size_in_bytes / (1024 * 1024)  # Convert to MB
#         print(f"Size of '{subfolder}': {size_in_mb:.2f} MB")


Size of 'raft-sft-output': 0.00 MB
Size of 'finetuned_raft_model': 71.03 MB
Size of '.virtual_documents': 0.00 MB
Size of 'wandb': 0.31 MB
Size of 'finetunedGemma': 34.17 MB


In [5]:
# import os

# # Define the path to the file
# file_path = '/kaggle/working/working_directory.zip'

# # Check if the file exists, then delete it
# if os.path.exists(file_path):
#     os.remove(file_path)
#     print(f"{file_path} has been deleted.")
# else:
#     print(f"{file_path} does not exist.")


/kaggle/working/working_directory.zip has been deleted.


In [32]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)

# disable_exllama=True

model = AutoModelForCausalLM.from_pretrained(output_dir_new,quantization_config=quantization_config_loading, device_map="auto")


Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--marcsun13--gemma-2-9b-it-GPTQ/snapshots/9c51e21691674594afc671d8edaeb93aa0cc9f7a/config.json
Model config Gemma2Config {
  "_name_or_path": "marcsun13/gemma-2-9b-it-GPTQ",
  "architectures": [
    "Gemma2ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "attn_logit_softcapping": 50.0,
  "bos_token_id": 2,
  "cache_implementation": "hybrid",
  "eos_token_id": 1,
  "final_logit_softcapping": 30.0,
  "head_dim": 256,
  "hidden_act": "gelu_pytorch_tanh",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 3584,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at marcsun13/gemma-2-9b-it-GPTQ were not used when initializing Gemma2ForCausalLM: {'model.layers.35.self_attn.k_proj.bias', 'model.layers.28.self_attn.v_proj.bias', 'model.layers.38.mlp.up_proj.bias', 'model.layers.41.self_attn.k_proj.bias', 'model.layers.33.mlp.gate_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.19.self_attn.v_proj.bias', 'model.layers.24.self_attn.v_proj.bias', 'model.layers.6.self_attn.q_proj.bias', 'model.layers.20.mlp.gate_proj.bias', 'model.layers.2.self_attn.v_proj.bias', 'model.layers.31.mlp.up_proj.bias', 'model.layers.7.mlp.gate_proj.bias', 'model.layers.41.mlp.gate_proj.bias', 'model.layers.33.self_attn.o_proj.bias', 'model.layers.24.mlp.up_proj.bias', 'model.layers.14.self_attn.o_proj.bias', 'model.layers.11.mlp.up_proj.bias', 'model.layers.23.self_attn.q_proj.bias', 'model.layers.18.self_attn.o_proj.bias', 'model.layers.19.self_attn.q_proj.bias', 'model.layers.13.self_attn.k_proj.bias', 'model.layer

In [33]:
raw_datasets["test"][0]

{'text': '<|system|>\n<eos>\n<|user|>\n<DOCUMENT>Primary injection test sets\nODEN A and ODEN AT\nCreated to test low-voltage and primary breakers, ODEN \nis designed to generate extraordinarily high power with-\nout sacrificing portability. Outputs can range up to many \nthousand amperes, and thanks to a highly sophisticated \nmeasurement section, ODEN has a very broad range of  \napplications.\nINGVAR\nThis powerful test system is designed for primary injection \ntesting of  protective relay equipment and circuit breakers. \nThe system consists of  a control unit and a current unit. \n</DOCUMENT>\n<DOCUMENT>PF tests for shielded or \nsheathed cables and accessories are diagnostic testing methods for field testing of \nservice aged cable installations. For lossless insulation,  the cable capacitance (C ) \nper unit length can be calculated using the following formula: \n𝐶 = 2𝜋𝑘𝑒0 ln(𝑑𝑖\n𝑑𝑐\n) \nwhere \nk is the insulation dielectric constant \ne0 is the permittivity (capacitance) of a

## Use Query only for inference

In [34]:
import torch



# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating

messages = [

    {

        "role": "system",

        "content": "You are a RAFT model you give answer based on context. You also give reason for why this is the answer.",

    },

    {"role": "user", "content": "What did Paul Graham believe was the future of web development?"},

]



# prepare the messages for the model

input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")



# inference

outputs = model.generate(

        input_ids=input_ids,

        max_new_tokens=500,

        do_sample=True,

        temperature=0.7,

        top_k=50,

        top_p=0.95

)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


BackendCompilerFailed: backend='inductor' raised:
RuntimeError: Found Tesla P100-PCIE-16GB which is too old to be supported by the triton GPU compiler, which is used as the backend. Triton only supports devices of CUDA Capability >= 7.0, but your device is of CUDA capability 6.0

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True


## Use Query along with retreived context for inference. 

* This is taken from the test set we created earlier.

In [35]:
context_and_query = """

\n<DOCUMENT>And it wasn't, so I would.\n\nIn the summer of 2012 my mother had a stroke, and the cause turned out to be a blood clot caused by colon cancer. The stroke destroyed her balance, and she was put in a nursing home, but she really wanted to get out of it and back to her house, and my sister and I were determined to help her do it. I used to fly up to Oregon to visit her regularly, and I had a lot of time to think on those flights. </DOCUMENT>\n<DOCUMENT>I had no idea. </DOCUMENT>\n<DOCUMENT>Around this time, in the spring of 2000, I had an idea. It was clear from our experience with Viaweb that web apps were the future. Why not build a web app for making web apps? Why not let people edit code on our server through the browser, and then host the resulting applications for them? [9] You could run all sorts of services on the servers that these applications could use just by making an API call: making and receiving phone calls, manipulating images, taking credit card payments, etc.\n\nI got so excited about this idea that I couldn't think about anything else. </DOCUMENT>\n

What did Paul Graham believe was the future of web development?

"""

In [36]:
context_and_query = """

\n<DOCUMENT> Emma is a creative and driven designer known for her innovative approach to user experience. With a keen eye for detail and a background in psychology, she combines aesthetics with usability, crafting interfaces that feel intuitive and engaging. Colleagues admire her collaborative spirit and problem-solving skills, as she thrives in fast-paced environments and enjoys tackling complex design challenges. Outside of work, Emma is passionate about photography and often finds inspiration in nature, incorporating earthy tones and organic shapes into her projects. Her dedication to continuous learning and user-centered design has earned her a respected place in her field. </DOCUMENT>\n<DOCUMENT>I had no idea. </DOCUMENT>\n<DOCUMENT>James is an insightful engineer with a knack for simplifying complex systems. With a foundation in mechanical engineering and a passion for sustainable technology, he’s developed solutions that enhance efficiency and reduce environmental impact. Known for his meticulous approach, James excels at troubleshooting and enjoys mentoring younger engineers, helping them navigate intricate technical challenges. Outside of work, he’s an avid cyclist and enjoys restoring vintage bikes, blending his technical skills with his love for craftsmanship. James’s commitment to innovation and teamwork makes him a valued member of his team and a respected figure in his industry.</DOCUMENT>\n

Why James is a valued member?

"""

In [37]:
import torch



# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating

messages = [

    {

        "role": "system",

        "content": "You are a RAFT model you give answer based on context. You also give reason for why this is the answer.",

    },

    {"role": "user", "content": context_and_query},

]



# prepare the messages for the model

input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")



# inference

outputs = model.generate(

        input_ids=input_ids,

        max_new_tokens=500,

        do_sample=True,

        temperature=0.01,

        top_k=50,

        top_p=0.95

)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


BackendCompilerFailed: backend='inductor' raised:
RuntimeError: Found Tesla P100-PCIE-16GB which is too old to be supported by the triton GPU compiler, which is used as the backend. Triton only supports devices of CUDA Capability >= 7.0, but your device is of CUDA capability 6.0

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True


# Reference:

1. https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Mistral/Supervised_fine_tuning_(SFT)_of_an_LLM_using_Hugging_Face_tooling.ipynb

# RAG

In [21]:
# from time import time
# import transformers
# import torch

# time_start = time()
# query_pipeline = transformers.pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         torch_dtype=torch.float16,
#         max_new_tokens=500,
#         temperature=0.1,
#         #max_length=1024,
#         device_map="auto",)
# time_end = time()
# print(f"Prepare pipeline: {round(time_end-time_start, 3)} sec.")

Prepare pipeline: 0.001 sec.


In [23]:
# %%capture
# !pip install -q nltk==3.9.1 --force-reinstall --no-cache-dir
# !pip install pandas langchain langchain_experimental langchain-groq
# !pip install langchain-chroma langchain-text-splitters langchain-huggingface 
# !pip install "unstructured[all-docs]"
# !pip install -q unstructured
# !apt-get install -y -q poppler-utils
# !pip install unstructured-client watermark
# !pip install reportlab
# !pip install PyPDF2
# !pip install pymupdf pdfplumber

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [24]:
# # Warning control
# !pip show unstructured

# import os
# import base64
# import nltk
# print(nltk.__version__)
# import pydantic
# print(pydantic.__version__)

# nltk.download('punkt_tab')
# import warnings
# warnings.filterwarnings('ignore')
# from IPython.display import JSON
# import json
# from unstructured.partition.html import partition_html
# from unstructured.partition.pdf import partition_pdf
# from unstructured.staging.base import dict_to_elements, elements_to_json
# import unstructured.partition
# from unstructured.partition.pdf import partition_pdf
# from io import StringIO 
# from lxml import etree
# from langchain_groq import ChatGroq
# from langchain_core.documents import Document
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from groq import Groq
# from IPython.display import Image, display
# import glob
# from langchain.schema import Document
# from reportlab.lib.pagesizes import letter
# from reportlab.pdfgen import canvas
# from groq import Groq
# from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
# from langchain_text_splitters import CharacterTextSplitter
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_chroma import Chroma
# from langchain_groq import ChatGroq
# from langchain.chains import RetrievalQA
# import PyPDF2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: unstructured
Version: 0.16.5
Summary: A library that prepares raw documents for downstream ML tasks.
Home-page: https://github.com/Unstructured-IO/unstructured
Author: Unstructured Technologies
Author-email: devops@unstructuredai.io
License: Apache-2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: backoff, beautifulsoup4, chardet, dataclasses-json, emoji, filetype, html5lib, langdetect, lxml, nltk, numpy, psutil, python-iso639, python-magic, python-oxmsg, rapidfuzz, requests, tqdm, typing-extensions, unstructured-client, wrapt
Required-by: 
3.9.1
2.9.2


[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [25]:
# pdf_path = "/kaggle/input/testing-manual/Transformer Testing Manual.pdf"

# # Load the PDF using UnstructuredPDFLoader
# loader = UnstructuredPDFLoader(pdf_path)
# documents = loader.load()

# text_splitter = CharacterTextSplitter(
#     chunk_size=2000,
#     chunk_overlap=500
# )

# text_chunks = text_splitter.split_documents(documents)

In [33]:
# persist_directory = "doc_db"
# embedding = HuggingFaceEmbeddings()

# vectorstore = Chroma.from_documents(
#     documents=text_chunks,
#     embedding=embedding,
#     persist_directory=persist_directory
# )

# retriever = vectorstore.as_retriever()

/tmp/ipykernel_30/872483988.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings()
/tmp/ipykernel_30/872483988.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()


In [27]:
# import sys
# from torch import cuda, bfloat16
# import torch
# import transformers
# from transformers import AutoTokenizer
# from time import time
# import chromadb
# from chromadb.config import Settings
# from langchain.llms import HuggingFacePipeline
# from langchain.document_loaders import PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.chains import RetrievalQA
# from langchain.vectorstores import Chroma

In [28]:
# llm = HuggingFacePipeline(pipeline=query_pipeline)

/tmp/ipykernel_30/725864647.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=query_pipeline)


In [32]:
# from langchain.prompts import PromptTemplate
# from langchain.chains import RetrievalQA

# template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, say that the question is outside of the given document, then answer the question from your knowledge. If the question is from the context, answer as normal. 

# Context: {context}
# Question: {question}
# Answer:"""

# QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# # Create a RetrievalQA chain with the custom prompt
# qa_chain_HF = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
#     #return_source_documents=True
# )

# question = "What can be done if TTR test set cannot be used"

# # Start the timer
# time_start = time()
# # Run the QA chain to get the answer
# result = qa_chain_HF({"query": question})
# # End the timer
# time_end = time()

# # Calculate the time taken to answer the question
# elapsed_time = time_end - time_start

# # Print the result and time taken
# print(result["result"])
# print(f"Time taken: {elapsed_time:.4f} seconds")


Use the following pieces of context to answer the question at the end. If you don't know the answer, say that the question is outside of the given document, then answer the question from your knowledge. If the question is from the context, answer as normal. 

Context: Voltage ratio is proportionally equal to turns ratio.

The TTR test gives the following:

It checks the turns ratio and polarity of single- and three-phase power

transformers, one phase at a time.

It verifies nameplate ratio, polarity, and vectors.

It checks polarity and the ratio (but not voltage rating) of transformers without

markings. Tests consider all transformer no-load tap positions. Tests consider

all load taps on load, tap changer (LTC) transformers if connected for voltage

ratio control. On LTC transformers connected for phase angle control, ratio

and polarity are completed only in neutral positions. If checked on load taps,

measurements may be taken for reference for future comparison, but will

deviat